# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [2]:
# Run this code:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [3]:
# Your code here
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [4]:
salaries.columns


Index(['Name', 'Job Titles', 'Department', 'Full or Part-Time',
       'Salary or Hourly', 'Typical Hours', 'Annual Salary', 'Hourly Rate'],
      dtype='object')

# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [5]:
# Función para calcular salario por hora
def calcular_salario_por_hora(row):
    if row['Salary or Hourly'] == 'Salary':
        annual_salary = row['Annual Salary']
        if pd.notna(annual_salary):
            return annual_salary / 2080  # considerando 52 semanas en un año y 40 horas por semana.
        else:
            return None
    elif row['Salary or Hourly'] == 'Hourly':
        return row['Hourly Rate']
    else:
        return None

# Aplicar la función a cada fila del DataFrame
salaries['Salario por Hora Calculado'] = salaries.apply(calcular_salario_por_hora, axis=1)

# Mostrar el resultado
print(salaries)

                        Name                              Job Titles  \
0          AARON,  JEFFERY M                                SERGEANT   
1            AARON,  KARINA   POLICE OFFICER (ASSIGNED AS DETECTIVE)   
2        AARON,  KIMBERLEI R                CHIEF CONTRACT EXPEDITER   
3        ABAD JR,  VICENTE M                       CIVIL ENGINEER IV   
4          ABASCAL,  REECE E             TRAFFIC CONTROL AIDE-HOURLY   
...                      ...                                     ...   
33178  ZYLINSKA,  KATARZYNA                           POLICE OFFICER   
33179     ZYMANTAS,  LAURA C                          POLICE OFFICER   
33180      ZYMANTAS,  MARK E                          POLICE OFFICER   
33181    ZYRKOWSKI,  CARLO E                          POLICE OFFICER   
33182   ZYSKOWSKI,  DARIUSZ                  CHIEF DATA BASE ANALYST   

             Department Full or Part-Time Salary or Hourly  Typical Hours  \
0                POLICE                 F           Salary

In [6]:
print(salaries.isnull().sum())

Name                              0
Job Titles                        0
Department                        0
Full or Part-Time                 0
Salary or Hourly                  0
Typical Hours                 25161
Annual Salary                  8022
Hourly Rate                   25161
Salario por Hora Calculado        0
dtype: int64


In [7]:
# Your code here
from scipy.stats import ttest_1samp

# Parámetro bajo prueba (valor de referencia)
valor_referencia = 30

# Extraer los datos de la columna creada
salarios_por_hora = salaries['Salario por Hora Calculado']

# Realizar la prueba de hipótesis de una muestra (two-sided)
t_statistic, p_valor = ttest_1samp(salarios_por_hora, valor_referencia)

# Imprimir resultados
print(f"Estadístico t: {t_statistic:.4f}")
print(f"Valor p: {p_valor:.4f}")

# Interpretar los resultados
alpha = 0.05  # Nivel de significancia
if p_valor < alpha:
    print("Se rechaza la hipótesis nula: El salario por hora es significativamente diferente de $30/hora.")
else:
    print("No se puede rechazar la hipótesis nula: No hay suficiente evidencia para concluir que el salario por hora es diferente de $30/hora.")


Estadístico t: 154.1767
Valor p: 0.0000
Se rechaza la hipótesis nula: El salario por hora es significativamente diferente de $30/hora.


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [8]:
# Your code here
from scipy import stats

# Calcular la media muestral y el error estándar de la media muestral
media_muestral = np.mean(salarios_por_hora)
std_error = stats.sem(salarios_por_hora)

# Calcular el intervalo de confianza del 95% utilizando t.interval
confidence_level = 0.95
degrees_freedom = len(salarios_por_hora) - 1

intervalo_confianza = stats.t.interval(confidence_level, df=degrees_freedom, loc=media_muestral, scale=std_error)

# Mostrar el resultado
print(f"Intervalo de Confianza del 95% para el salario por hora medio:")
print(f"({intervalo_confianza[0]:.2f}, {intervalo_confianza[1]:.2f})")


Intervalo de Confianza del 95% para el salario por hora medio:
(39.44, 39.69)


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [9]:
# Your code here
from statsmodels.stats.proportion import proportions_ztest

# Calcular el número de trabajadores por hora (éxitos) y el tamaño de la muestra
num_success = salaries['Salary or Hourly'].value_counts()['Hourly']
total_workers = len(salaries)

# Parámetros para la prueba de proporciones
null_proportion = 0.25  # Proporción bajo la hipótesis nula (25%)
alpha = 0.05  # Nivel de significancia del 95%

# Realizar la prueba de proporciones usando proportions_ztest
z_stat, p_value = proportions_ztest(num_success, total_workers, null_proportion)

# Imprimir resultados
print(f"Estadístico Z: {z_stat:.4f}")
print(f"Valor p: {p_value:.4f}")

# Interpretar los resultados
if p_value < alpha:
    print("Se rechaza la hipótesis nula: La proporción de trabajadores por hora es significativamente diferente del 25%.")
else:
    print("No se puede rechazar la hipótesis nula: No hay suficiente evidencia para concluir que la proporción de trabajadores por hora es diferente del 25%.")


Estadístico Z: -3.5100
Valor p: 0.0004
Se rechaza la hipótesis nula: La proporción de trabajadores por hora es significativamente diferente del 25%.
